In [3]:
import numpy as np
import pandas as pd


# Mock patient data and the modifications to the master spec
patient_data = pd.DataFrame({
    'patient_id': [1, 2],
    'age': [25, 70],
    'sex': ['M', 'F'],
    'Obese': [1, 0],
    'Hypertension': [0, 1],
    'Atrial_Fibrillation': [1, 0]
})


# Base transition matrix
basis_transition_matrix = {
    "states": [
        "Neutral", "Cardiac Ischaemia", "Sepsis", "Acute Anxiety/Panic",
        "Breathing Difficulty", "Hypovolaemia", "Arrhythmic Flare", "Hypoglycemia",
        "TIA", "Bathroom (harmless)", "White Coat Syndrome (harmless)", "STEMI (crisis)",
        "Septic Shock (crisis)", "Compromised Airway (crisis)", "Haemorrhagic Shock (crisis)",
        "Stroke (crisis)", "Death"
    ],
    "transition_matrix": [
        [0.70, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.05, 0.05, 0, 0, 0, 0, 0, 0],
        [0.50, 0.30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.20, 0, 0, 0, 0, 0],
        [0.15, 0, 0.45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.40, 0, 0, 0, 0],
        [0.80, 0, 0, 0.20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0.50, 0, 0, 0, 0.35, 0, 0, 0, 0, 0, 0, 0, 0.15, 0, 0, 0, 0],
        [0.35, 0, 0, 0, 0, 0.40, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0],
        [0.65, 0, 0, 0, 0, 0, 0.20, 0, 0, 0, 0, 0.15, 0, 0, 0, 0, 0],
        [0.75, 0, 0, 0, 0, 0, 0, 0.17, 0, 0, 0, 0, 0, 0.08, 0, 0, 0],
        [0.70, 0, 0, 0, 0, 0, 0, 0, 0.10, 0, 0, 0, 0, 0, 0, 0.20, 0],
        [1.00, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1.00, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0.80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.20],
        [0, 0, 0.90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.10],
        [0, 0, 0, 0, 0, 0, 0, 0.50, 0, 0, 0, 0, 0, 0, 0, 0, 0.50],
        [0, 0, 0, 0, 0, 0.70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.30],
        [0, 0, 0, 0, 0, 0, 0, 0, 0.85, 0, 0, 0, 0, 0, 0, 0, 0.15],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.00]
    ]
}


# Define the MASTER_SPEC as described in the user's scenario
MASTER_SPEC = {
    "Age_Pediatric": {"risk_modifiers": {"sepsis_warning": 1.0, "pre_mi_warning": 0.5}},
    "Age_Elderly": {"risk_modifiers": {"pre_mi_warning": 1.5, "sepsis_warning": 1.5}},
    "Sex_Male": {"risk_modifiers": {"pre_mi_warning": 1.2}},
    "Sex_Female": {"risk_modifiers": {"pre_mi_warning": 1.0}},
    "Obese": {"risk_modifiers": {"pre_mi_warning": 2.0, "stemi_crisis": 1.5, "sepsis_warning": 1.0}},
    "Underweight": {"risk_modifiers": {"infection_warning": 1.3}},
    "Hypertension": {"risk_modifiers": {"pre_mi_warning": 2.0, "stemi_crisis_if_in_pre_mi_warning": 1.5}},
    "Atrial_Fibrillation": {"risk_modifiers": {"pre_mi_warning": 1.2, "stroke_crisis": 2.0}},
    "Diabetes_Type2": {"risk_modifiers": {"hypoglycemia_warning": 1.5}},
    "Seizure_Disorder": {"risk_modifiers": {"seizure_crisis": 1.5}},
    "Depression_Anxiety": {"risk_modifiers": {"panic_warning": 2.0}},
    "COPD": {"risk_modifiers": {"breathing_difficulty_warning": 1.5, "compromised_airway_crisis_if_in_breathing_difficulty": 1.3}},
    "Hemophilia": {"risk_modifiers": {"hypovolemia_warning": 2.0, "hemorrhagic_crisis_if_in_hypovolemia": 2.0}},
    "Anemia": {"risk_modifiers": {"hemorrhagic_crisis_if_in_hypovolemia": 1.5, "breathing_difficulty_warning": 1.2}},
    "Chronic_Kidney_Disease": {"risk_modifiers": {"sepsis_warning": 1.2, "hemorrhagic_crisis_if_in_hypovolemia": 1.2}},
    "MildInjury_Pain": {},
    "ChestPain": {"risk_modifiers": {"cardiac_ischaemia_warning": 0.8}},
    "Asthma": {"risk_modifiers": {"breathing_difficulty_warning": 1.0}},
    "Appendicitis": {"risk_modifiers": {"sepsis_warning": 1.3}},
    "Drug_Overdose": {"risk_modifiers": {"breathing_difficulty_warning": 1.5}},
}

state_mapping = {
    "sepsis_warning": "Sepsis",
    "pre_mi_warning": "Cardiac Ischaemia",
    "stemi_crisis": "STEMI (crisis)",
    "hypoglycemia_warning": "Hypoglycemia",
    "seizure_crisis": "TIA",
    "panic_warning": "Acute Anxiety/Panic",
    "breathing_difficulty_warning": "Breathing Difficulty",
    "compromised_airway_crisis_if_in_breathing_difficulty": "Compromised Airway (crisis)",
    "hypovolemia_warning": "Hypovolaemia",
    "hemorrhagic_crisis_if_in_hypovolemia": "Haemorrhagic Shock (crisis)",
    "stroke_crisis": "Stroke (crisis)"
}


def create_patient_specific_matrices(basis_transition_matrix, patient_data, master_spec, state_mapping):
    states = basis_transition_matrix['states']
    base_matrix = np.array(basis_transition_matrix['transition_matrix'])
    
    patient_matrices = {}
    
    for index, patient in patient_data.iterrows():
        patient_matrix = np.copy(base_matrix)
        
        for characteristic, details in master_spec.items():
            if patient.get(characteristic, 0) == 1 and "risk_modifiers" in details:
                for modified_state, modifier in details['risk_modifiers'].items():
                    if modified_state in state_mapping and state_mapping[modified_state] in states:
                        from_state_index = states.index(state_mapping[modified_state])
                        for to_state_index, _ in enumerate(states):
                            patient_matrix[from_state_index, to_state_index] *= modifier
        
        patient_matrices[patient['patient_id']] = patient_matrix
    
    return patient_matrices


# Applying the function with the mapping
patient_matrices = create_patient_specific_matrices(basis_transition_matrix, patient_data, MASTER_SPEC, state_mapping)
patient_matrices


{1: array([[0.7  , 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025,
         0.05 , 0.05 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [1.2  , 0.72 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.48 , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.15 , 0.   , 0.45 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.4  , 0.   , 0.   , 0.   , 0.   ],
        [0.8  , 0.   , 0.   , 0.2  , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.5  , 0.   , 0.   , 0.   , 0.35 , 0.   , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.15 , 0.   , 0.   , 0.   , 0.   ],
        [0.35 , 0.   , 0.   , 0.   , 0.   , 0.4  , 0.   , 0.   , 0.   ,
         0.   , 0.   , 0.   , 0.   , 0.   , 0.25 , 0.   , 0.   ],
        [0.65 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.2  , 0.   , 0.   ,
         0.   , 0.   , 0.15 , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.75 , 0.   , 0.   , 0